- age
    - younger vs. older people and decile scores, hypothesize with equivalent means
    - age-specific, using percentage of those who reoffend and see where each age compares to the mean of all people
    
- decile score

- years between offenses

- race
    - compared with decile score in more depth




In [190]:
import numpy as np
import pandas as pd

from IPython.display import Image

import warnings
warnings.simplefilter("ignore")

In [191]:
compas = pd.read_csv("Compas.csv")

In [192]:
compas.head()

,Unnamed: 0,id,first,last,sex,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,is_recid,is_violent_recid,v_decile_score,v_score_text,decile_score.1,score_text
0,0,1,miguel,hernandez,Male,69,Greater than 45,Other,0,1,0,0,0,0,0,1,Low,1,Low
1,1,2,michael,ryan,Male,31,25 - 45,Caucasian,0,5,0,0,0,-1,0,2,Low,5,Medium
2,2,3,kevon,dixon,Male,34,25 - 45,African-American,0,3,0,0,0,1,1,1,Low,3,Low
3,3,4,ed,philo,Male,24,Less than 25,African-American,0,4,0,1,4,1,0,3,Low,4,Low
4,4,5,marcu,brown,Male,23,Less than 25,African-American,0,8,1,0,1,0,0,6,Medium,8,High


In [193]:
compas.drop(compas.columns[0], axis = 1)

,id,first,last,sex,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,is_recid,is_violent_recid,v_decile_score,v_score_text,decile_score.1,score_text
0,1,miguel,hernandez,Male,69,Greater than 45,Other,0,1,0,0,0,0,0,1,Low,1,Low
1,2,michael,ryan,Male,31,25 - 45,Caucasian,0,5,0,0,0,-1,0,2,Low,5,Medium
2,3,kevon,dixon,Male,34,25 - 45,African-American,0,3,0,0,0,1,1,1,Low,3,Low
3,4,ed,philo,Male,24,Less than 25,African-American,0,4,0,1,4,1,0,3,Low,4,Low
4,5,marcu,brown,Male,23,Less than 25,African-American,0,8,1,0,1,0,0,6,Medium,8,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11752,11753,patrick,hamilton,Male,47,Greater than 45,Other,0,3,0,0,1,0,0,1,Low,3,Low
11753,11754,raymond,hernandez,Male,22,Less than 25,Caucasian,0,7,3,5,3,1,0,5,Medium,7,Medium
11754,11755,dieuseul,pierre-gilles,Male,35,25 - 45,Other,0,4,0,0,0,0,0,3,Low,4,Low
11755,11756,scott,lomagistro,Male,29,25 - 45,Caucasian,0,3,0,0,2,0,0,2,Low,3,Low


In [194]:
young = compas[compas.age_cat == "Less than 25"].v_decile_score
middle = compas[compas.age_cat == "25 - 45"].v_decile_score
old = compas[compas.age_cat == "Greater than 45"].v_decile_score

In [195]:
from statsmodels.stats.weightstats import ttest_ind
from scipy import stats

These tests are just checking if one age group has a higher risk of violence than the other

In [196]:
print(young.mean(), middle.mean(), old.mean())

6.049180327868853 3.3771995788840425 1.7897301349325336


In [197]:
tstat, pvalue, ddof = ttest_ind(
                                young,
                                middle,
                                alternative="larger",
                                usevar="unequal"
)
print(tstat, pvalue, ddof)

56.21398274904184 0.0 5172.47658225915


In [198]:
 tstat, pvalue, ddof = ttest_ind(
                                middle,
                                old,
                                alternative="larger",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof)    

38.63847158703123 5.2422454240000335e-297 7127.205606761121


In [199]:
tstat, pvalue, ddof = ttest_ind(
                                young,
                                old,
                                alternative="larger",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

87.24747908351314 0.0 4723.302717729157


In [200]:
highrisk = compas[compas.score_text == "High"]
highrisk = highrisk[(highrisk.is_recid != -1) & (highrisk.decile_score != -1)]
## -1 = unknown, so those were deleted

In [201]:
highrisk["accuracy"] = (highrisk.is_recid == 1)
##checked if is_recid is True with high risk decile score to determine accuracy

In [202]:
young2 = highrisk[highrisk.age_cat == "Less than 25"].accuracy
middle2 = highrisk[highrisk.age_cat == "25 - 45"].accuracy
old2 = highrisk[highrisk.age_cat == "Greater than 45"].accuracy
##subset by age

In [203]:
print(young2.mean(), middle2.mean(), old2.mean())

0.5481927710843374 0.5325732899022801 0.4251207729468599


In [204]:
 tstat, pvalue, ddof = ttest_ind(
                                young2,
                                middle2,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

0.6505547751165384 0.5154436817845576 1361.6788657477787


In [205]:
 tstat, pvalue, ddof = ttest_ind(
                                young2,
                                old2,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

3.116051265304813 0.001986434026815587 345.5133605030412


In [206]:
 tstat, pvalue, ddof = ttest_ind(
                                middle2,
                                old2,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

2.882869983424161 0.004244991793179706 281.10125290631083


These hypothesis tests were checking if there was a significant difference between the age groups accuracy. 
It seems as though there is a signifcant difference between the younger two groups and the oldest group.
It would also be a good idea to look at low risk. 

In [207]:
##same procedure but with low risk
lowrisk = compas[compas.score_text == "Low"]
lowrisk = lowrisk[(lowrisk.is_recid != -1) & (lowrisk.decile_score != -1)]

In [208]:
lowrisk["accuracy"] = (lowrisk.is_recid == 0)
##accurate if is_recid == 0

In [209]:
young3 = lowrisk[lowrisk.age_cat == "Less than 25"].accuracy
middle3 = lowrisk[lowrisk.age_cat == "25 - 45"].accuracy
old3 = lowrisk[lowrisk.age_cat == "Greater than 45"].accuracy

In [210]:
print(young3.mean(), middle3.mean(), old3.mean())

0.690734055354994 0.7607060185185185 0.8178879310344828


In [211]:
 tstat, pvalue, ddof = ttest_ind(
                                young3,
                                middle3,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

-3.9737509848699326 7.499325893430174e-05 1192.5917089874865


In [212]:
 tstat, pvalue, ddof = ttest_ind(
                                young3,
                                old3,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

-6.919646610619877 6.9337001458953706e-12 1369.0461762834632


In [213]:
 tstat, pvalue, ddof = ttest_ind(
                                middle3,
                                old3,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

-4.958642361248988 7.383646858476191e-07 4132.705113743328


The low risk accuracy for the oldest group is much higher.
The differences in accuracy are significant between all groups. They likely do not have the same average accuracy. 
Since the average accuracy for the oldest group is significantly lower for high risk and higher for low risk, this may mean this algorithm may give the older group lower scores in general. 

In [281]:
highriskrace = compas[compas.score_text == "High"]
highriskrace = highriskrace[(highriskrace.is_recid != -1) & (highriskrace.decile_score != -1)]
highriskrace["accuracyhigh"] = (highriskrace.is_recid == 1)

lowriskrace = compas[compas.score_text == "Low"]
lowriskrace = lowriskrace[(lowriskrace.is_recid != -1) & (lowriskrace.decile_score != -1)]
lowriskrace["accuracylow"] = (lowriskrace.is_recid == 0)

In [282]:
hrr = highriskrace[(highriskrace.race == "African-American") |(highriskrace.race == "Caucasian") ]
hrr = hrr[(hrr.is_recid != -1) & (hrr.decile_score != -1)]

lrr = lowriskrace[(lowriskrace.race == "African-American") |(lowriskrace.race == "Caucasian") ]
lrr = lrr[(LCo2.is_recid != -1) & (lrr.decile_score != -1)]

In [283]:
HWhitt = hrr[(hrr.race=="Caucasian")&(hrr.score_text=="High")].accuracyhigh
HWhitemen = hrr[(hrr.race=="Caucasian")&(hrr.sex=="Male")&(hrr.score_text=="High")].accuracyhigh
HWhitewomen = hrr[(hrr.race=="Caucasian")&(hrr.sex=="Female")&(hrr.score_text=="High")].accuracyhigh
HBlakk = hrr[(hrr.race=="African-American")&(hrr.score_text=="High")].accuracyhigh
HBlackmen = hrr[(hrr.race=="African-American")&(hrr.sex=="Male")&(hrr.score_text=="High")].accuracyhigh
HBlackwomen = hrr[(hrr.race=="African-American")&(hrr.sex=="Female")&(hrr.score_text=="High")].accuracyhigh

LWhitt = lrr[(lrr.race=="Caucasian")&(lrr.score_text=="Low")].accuracylow
LWhitemen = lrr[(lrr.race=="Caucasian")&(lrr.sex=="Male")&(lrr.score_text=="Low")].accuracylow
LWhitewomen = lrr[(lrr.race=="Caucasian")&(lrr.sex=="Female")&(lrr.score_text=="Low")].accuracylow
LBlakk = lrr[(lrr.race=="African-American")&(lrr.score_text=="Low")].accuracylow
LBlackmen = lrr[(lrr.race=="African-American")&(lrr.sex=="Male")&(lrr.score_text=="Low")].accuracylow
LBlackwomen = lrr[(lrr.race=="African-American")&(lrr.sex=="Female")&(lrr.score_text=="Low")].accuracylow

In [284]:
tstat, pvalue, ddof = ttest_ind(HWhitt,HBlakk,alternative="smaller",usevar="unequal")
tstat1, pvalue1, ddof1 = ttest_ind(HWhitemen,HBlackmen,alternative="smaller",usevar="unequal")
tstat2, pvalue2, ddof2 = ttest_ind(HWhitewomen,HBlackwomen,alternative="smaller",usevar="unequal")
tstat3, pvalue3, ddof3 = ttest_ind(HBlackwomen,HWhitemen,alternative="smaller",usevar="unequal")
tstat4, pvalue4, ddof4 = ttest_ind(HWhitewomen,HBlackmen,alternative="smaller",usevar="unequal")
tstat5, pvalue5, ddof5 = ttest_ind(HWhitewomen,HWhitemen,alternative="smaller",usevar="unequal")
tstat6, pvalue6, ddof6 = ttest_ind(HBlackwomen,HBlackmen,alternative="smaller",usevar="unequal")

Ltstat, Lpvalue, Lddof = ttest_ind(LWhitt,LBlakk,alternative="larger",usevar="unequal")
Ltstat1, Lpvalue1, Lddof1 = ttest_ind(LWhitemen,LBlackmen,alternative="larger",usevar="unequal")
Ltstat2, Lpvalue2, Lddof2 = ttest_ind(LWhitewomen,LBlackwomen,alternative="larger",usevar="unequal")
Ltstat3, Lpvalue3, Lddof3 = ttest_ind(LBlackwomen,LWhitemen,alternative="larger",usevar="unequal")
Ltstat4, Lpvalue4, Lddof4 = ttest_ind(LWhitewomen,LBlackmen,alternative="larger",usevar="unequal")
Ltstat5, Lpvalue5, Lddof4 = ttest_ind(LWhitewomen,LWhitemen,alternative="larger",usevar="unequal")
Ltstat6, Lpvalue6, Lddof5 = ttest_ind(LBlackwomen,LBlackmen,alternative="larger",usevar="unequal")

In [285]:
print("High risk accuracy p-value", "\n","\n"
      "White and Black p-value:", round(pvalue, 4), "\n" 
      "White men and Black men p-value:", round(pvalue1, 4),"\n"
      "White women and Black women p-value:", round(pvalue2, 4),"\n"
      "Black women and White men p-value:", round(pvalue3, 4), "\n"
      "White women and Black men p-value:",round(pvalue4, 4), "\n"
      "White women and White men p-value:", round(pvalue5, 4), "\n"
      "Black women and Black men p-value:", round(pvalue6, 4), "\n" "\n"

      "Low risk accuracy p-value", "\n","\n"
      "White and Black p-value:", round(Lpvalue, 4), "\n" 
      "White men and Black men p-value:", round(Lpvalue1, 4),"\n"
      "White women and Black women p-value:", round(Lpvalue2, 4),"\n"
      "Black women and White men p-value:", round(Lpvalue3, 4), "\n"
      "White women and Black men p-value:", round(Lpvalue4, 4), "\n"
      "White women and White men p-value:", round(Lpvalue5, 4), "\n"
      "Black women and Black men p-value:", round(Lpvalue6, 4), "\n")


High risk accuracy p-value 
 
White and Black p-value: 0.0557 
White men and Black men p-value: 0.251 
White women and Black women p-value: 0.1496 
Black women and White men p-value: 0.033 
White women and Black men p-value: 0.0005 
White women and White men p-value: 0.0048 
Black women and Black men p-value: 0.0032 

Low risk accuracy p-value 
 
White and Black p-value: 0.0 
White men and Black men p-value: 0.0 
White women and Black women p-value: 0.5392 
Black women and White men p-value: 0.0031 
White women and Black men p-value: 0.0 
White women and White men p-value: 0.0035 
Black women and Black men p-value: 0.0 



# Findings

### There is a statistically significant discrepancy in the accuracies of "High" predictive values for recidivism when comparing:
- Black women and White men
- White women and Black men 
- White women and White men  
- Black women and Black men 

### There is a statistically significant discrepancy in the accuracies of "Low" predictive values for recidivism when comparing:
- White and Black overall population
- White men and Black men 
- White men and Black women 
- White women and Black men
- White women and White men
- Black women and Black men

In [242]:
highvisual = hrr.groupby(["race", "sex", "score_text"])[["accuracyhigh"]].mean()
lowvisual = lrr.groupby(["race", "sex", "score_text"])[["accuracylow"]].mean()
highvisual

accuracyhigh
race             sex    score_text              
African-American Female High            0.446701
                 Male   High            0.551416
Caucasian        Female High            0.384615
                 Male   High            0.530351

In [243]:
lowvisual

accuracylow
race             sex    score_text             
African-American Female Low            0.829175
                 Male   Low            0.698218
Caucasian        Female Low            0.826923
                 Male   Low            0.777157